In [4]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
import time
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [3]:
!pip install scikit-learn

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

ImportError: cannot import name '_OneToOneFeatureMixin' from 'sklearn.base' (C:\Softwares\AnacondaS\lib\site-packages\sklearn\base.py)

In [ ]:
df = pd.read_csv("C:/Users/jpadmanabhan/Downloads/Tweets.csv")
df

In [ ]:
(len(df) - df.count())/len(df) #To get % of missing val. if any col has >90%, drop them

In [ ]:
df.retweet_count.sort_values

In [ ]:
df = df.drop(['airline_sentiment_gold','negativereason_gold','tweet_coord'],axis=1)

In [ ]:
sentiment_count = df['airline_sentiment'].value_counts()
sentiment_count

In [ ]:
#Graphical representation of airline sentiment
sns.countplot(x='airline_sentiment',data=df,order=['negative','neutral','positive'])
plt.show()

In [ ]:
sns.catplot(x = 'airline_sentiment',data=df,
               order = ['negative','neutral','positive'],
               kind = 'count',col_wrap=3,col='airline',
               height=4,aspect=0.6,sharex=False,sharey=False)
plt.show()

In [ ]:
df['negativereason'].value_counts()

In [ ]:
#Graphical representation of negativereason towards airlines
sns.catplot(x = 'airline',data = df,kind = 'count',hue='negativereason',height=12,aspect=.9)
plt.show()

In [ ]:
sns.catplot(x = 'negativereason',data=df,kind='count',col='airline',height=9,aspect=.8,col_wrap=2,sharex=False,sharey=False)
plt.show()

In [ ]:
start = time.time()
#remove words which starts with @ symbols, * is for checking if the char starts with the specified val.
df['text'] = df['text'].map(lambda x:re.sub('@\w*','',str(x)))
#remove special characters except [a-zA-Z]
df['text'] = df['text'].map(lambda x:re.sub('[^a-zA-Z]',' ',str(x)))
#remove link starts with https
df['text'] = df['text'].map(lambda x:re.sub('http.*','',str(x)))
end = time.time()

In [ ]:
#To find total time consume to filter data
end - start

In [ ]:
df['text'].head()

In [ ]:
#Converting all text to lowercase because lowercase and uppercase char have differeny ascii nos.
df['text'] = df['text'].map(lambda x:str(x).lower())

In [ ]:
df['text'].head()

In [ ]:
# Remove stopwords from comments such as 'the', 'a', 'an', 'in', etc
# Not used PorterStemmer to make words pure
corpus = []

In [ ]:
sw_none = df['text'].map(lambda x:corpus.append(' '.join([word for word in str(x).strip().split() 
                                                          if not word in set(stopwords.words('english'))])))
sw_none

In [ ]:
# Run for better understanding of the logic
for word in str(df['text']).strip().split():
    if not word in set(stopwords.words('english')):
        print(word)

In [ ]:
corpus[:5]

In [ ]:
ps = PorterStemmer()
  
#sentence = "Programmers program with programming languages"
#words = word_tokenize(sentence)
stem_corpus = []
stem_corpus1 = []
for w in corpus:
    stem_corpus = stem_corpus1.append(ps.stem(w))
    #stem_corpus.append(ps.stem(w))

In [ ]:
st = LancasterStemmer()

stem_corpus = []
stem_corpus1 = []
for w in corpus:
    stem_corpus = stem_corpus1.append(st.stem(w))

In [ ]:
stem_corpus1[:5]

### Training and Testing

In [ ]:
x = pd.DataFrame(data=corpus,columns=['new_text'])
x

In [ ]:
y = df['airline_sentiment'].map({'neutral':1,'negative':0,'positive':2})
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
#Use TfidfVectorizer for feature extraction
#token_patten #2 for word length greater than 2>=
vector = TfidfVectorizer(stop_words='english', sublinear_tf=True,strip_accents='ascii',
                         analyzer='word',token_pattern=r'\w{2,}',ngram_range=(1,1),max_features=30000)

In [ ]:
x_train_word_feature = vector.fit_transform(x_train['new_text']).toarray()
x_test_word_feature = vector.transform(x_test['new_text']).toarray()

In [ ]:
x_train_word_feature.shape, x_test_word_feature.shape

In [ ]:
lr = LogisticRegression(max_iter=10000)
lr.fit(x_train_word_feature,y_train)

In [ ]:
y_predlr = lr.predict(x_test_word_feature)
accuracy = accuracy_score(y_test,y_predlr)
accuracy
#0.7687841530054644

In [ ]:
cm = confusion_matrix(y_test,y_predlr)
cm

In [ ]:
print(classification_report(y_test,y_predlr),'\n',accuracy)

In [ ]:
y_pred_prob = lr.predict_proba(x_train_word_feature) #Predict gives 0 or 1 as o/p, Predict_proba gives prob. of both 0 and 1.
y_pred_prob[:5]

In [ ]:
rfc = RandomForestClassifier(max_depth=3, random_state=0)
rfc.fit(x_train_word_feature,y_train)

In [ ]:
y_predrfc = rfc.predict(x_test_word_feature)
accuracy = accuracy_score(y_test,y_predrfc)
accuracy

In [ ]:
print(classification_report(y_test,y_predrfc),'\n',accuracy)

In [ ]:
xgb1 = xgb.XGBClassifier(learning_rate=0.30009999, max_depth=5)
xgb1.fit(x_train_word_feature, y_train)

In [ ]:
y_predxgb = xgb1.predict(x_test_word_feature)
accuracy = accuracy_score(y_test,y_predxgb)
accuracy

In [ ]:
print(classification_report(y_test,y_predxgb),'\n',accuracy)

In [ ]:
gnb = GaussianNB()
gnb.fit(x_train_word_feature, y_train)

In [ ]:
y_predgnb = gnb.predict(x_test_word_feature)
accuracy = accuracy_score(y_test,y_predgnb)
accuracy

In [ ]:
print(classification_report(y_test,y_predgnb),'\n',accuracy)

In [ ]:
svm1 = svm.SVC()
svm1.fit(x_train_word_feature, y_train)

In [ ]:
y_predsvm = svm1.predict(x_test_word_feature)
accuracy = accuracy_score(y_test,y_predsvm)
accuracy

In [ ]:
print(classification_report(y_test,y_predsvm),'\n',accuracy)

In [ ]:
dtc = tree.DecisionTreeClassifier()
dtc.fit(x_train_word_feature, y_train)

In [ ]:
y_preddtc = dtc.predict(x_test_word_feature)
accuracy = accuracy_score(y_test,y_preddtc)
accuracy

In [ ]:
print(classification_report(y_test,y_preddtc),'\n',accuracy)